In [1]:
import tensorflow as tf

2023-08-03 18:01:42.235566: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-03 18:01:43.166635: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import os,timeit
import numpy as np

In [3]:
from numpy import random

In [4]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [5]:
from sys import platform
DATA_DIRECTORY = ""
SLASH = ""
if platform == "linux" or platform == "linux2":
    DATA_DIRECTORY = "/home/tyagi/Desktop/wheat/data/BULK/"
    SLASH = "/"
elif platform == "win32":
    DATA_DIRECTORY = "D:\mvl\wheat\data\BULK\\"
    SLASH="\\"

In [6]:
#Constants
BAND_NUMBER = 60
FILLED_AREA_RATIO = 0.90
TOTAL_IMAGE_COUNT = 2400
IMAGE_COUNT = int(TOTAL_IMAGE_COUNT/4)
NUM_VARIETIES = 4
NUM_OF_BANDS = 15

REMOVE_NOISY_BANDS = False
FIRST_BAND = 15
LAST_BAND = 161

IMAGE_WIDTH = 30
IMAGE_HEIGHT = 30

In [7]:
from enum import Enum

class filter_method(Enum):
    none = 0
    snv = 1
    msc = 2
    savgol = 3
    
FILTER = filter_method(1).name

# to be set if filter chosen is savgol
WINDOW = 7
ORDER = 2
DERIVATIVE = "none"

In [8]:
from enum import Enum
 
class feature_extraction_method(Enum):
    none = 0
    pca_loading = 1
    lda = 2
    ipca = 3

FEATURE_EXTRACTION = feature_extraction_method(0).name

NUM_OF_BANDS = 3
if FEATURE_EXTRACTION == "pca_loading" or FEATURE_EXTRACTION == "ipca":
    NUM_OF_BANDS = 8
elif FEATURE_EXTRACTION == "lda":
    NUM_OF_BANDS = 3
    assert NUM_OF_BANDS <= min(NUM_VARIETIES-1,168),"NUM_OF_BANDS is greater."

In [9]:
# List for All varieties
VARIETIES = []
VARIETIES_CODE = {}

for name in os.listdir(DATA_DIRECTORY):
    if (name.endswith(".hdr") or name.endswith(".bil")):
        continue
    VARIETIES_CODE[name] = len(VARIETIES)
    VARIETIES.append(name)
    if len(VARIETIES)==NUM_VARIETIES:
        break

In [10]:
def start_timer():
    print("Testing started")
    return timeit.default_timer()

def end_timer():
    return timeit.default_timer()

def show_time(tic,toc): 
    test_time = toc - tic
    print('Testing time (s) = ' + str(test_time) + '\n')

In [11]:
def dataset_file_name(variety):
    name = "./dataset/V"+str(variety).zfill(3)+"_IC_"+str(TOTAL_IMAGE_COUNT).zfill(5)+"_FilledArea_"+str(FILLED_AREA_RATIO)+"_NumOfBands_"+str(NUM_OF_BANDS)+"_FB_"+str(FIRST_BAND)+"_LB_"+str(LAST_BAND)+"_BandNo_"+str(BAND_NUMBER)+"_ImageHeight_"+str(IMAGE_HEIGHT)+"_ImageWidth_"+str(IMAGE_WIDTH)+"_FILTER_"+str(FILTER)+"_FeatureExtraction_"+str(FEATURE_EXTRACTION)
    if REMOVE_NOISY_BANDS:
        name+="_REMOVE_NOISY_BANDS_"+str(REMOVE_NOISY_BANDS)
    if FILTER == "savgol":
        name+="_WINDOW_"+str(WINDOW)+"_ORDER_"+str(ORDER)
    return name

In [12]:
train_dataset = []
train_dataset_label = []
test_dataset=[]
test_dataset_label = []

for idx, v in enumerate(VARIETIES):
    print("idx: ",idx)
    if idx >= NUM_VARIETIES:
        break
    train_dataset= train_dataset + np.load(dataset_file_name(v)+"_train_dataset.npy").tolist()
    train_dataset_label = train_dataset_label + np.load(dataset_file_name(v)+"_train_dataset_label.npy").tolist()
    test_dataset = test_dataset + np.load(dataset_file_name(v)+"_test_dataset.npy").tolist()
    test_dataset_label = test_dataset_label + np.load(dataset_file_name(v)+"_test_dataset_label.npy").tolist()
    
train_dataset = np.array(train_dataset)
train_dataset_label = np.array(train_dataset_label)
test_dataset = np.array(test_dataset)
test_dataset_label = np.array(test_dataset_label)

idx:  0
idx:  1
idx:  2
idx:  3


In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [14]:
nsamples, nx, ny,nb = train_dataset.shape
nsamplestest, nx, ny,nb = test_dataset.shape
d2_train_dataset = train_dataset.reshape((nsamples,nx*ny*nb))
d2_test_dataset = test_dataset.reshape((nsamplestest,nx*ny*nb))

In [ ]:
# Flatten the images to 2D for Naive Bayes implementation
X_train_flat = train_dataset.reshape(-1, 30 * 30 * 168)
X_test_flat = test_dataset.reshape(-1, 30 * 30 * 168)

# Create TensorFlow placeholders for input data and labels
input_images = tf.placeholder(tf.float32, shape=(None, 30 * 30 * 168))
input_labels = tf.placeholder(tf.int32, shape=(None,))

# Calculate the prior probabilities
unique_labels, label_counts = np.unique(train_dataset_label, return_counts=True)
prior_probs = label_counts / len(train_dataset_label)

# Calculate the likelihoods assuming Gaussian distribution for each feature and each class
num_classes = 4
likelihoods = []
for label in range(num_classes):
    label_images = X_train_flat[train_dataset_label == label]
    label_likelihood = tf.reduce_mean(label_images, axis=0)
    likelihoods.append(label_likelihood)

# Convert likelihoods to TensorFlow tensors
likelihoods = tf.convert_to_tensor(likelihoods)

# Classify new images using the Naive Bayes classifier
log_likelihoods = tf.math.log(likelihoods)
log_priors = tf.math.log(prior_probs)
log_posteriors = log_likelihoods + log_priors
predictions = tf.argmax(log_posteriors, axis=0)

# Calculate accuracy for evaluation
correct_predictions = tf.equal(predictions, input_labels)
accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))

# Training
learning_rate = 0.01
num_epochs = 50
batch_size = 32

# Define the loss function (e.g., cross-entropy) and optimizer (e.g., gradient descent)
loss = ...  # Define your loss function here
optimizer = ...  # Define your optimizer here

# Create a TensorFlow session
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_batches = len(X_train_flat) // batch_size

    for epoch in range(num_epochs):
        avg_loss = 0.0
        for batch in range(num_batches):
            start_idx = batch * batch_size
            end_idx = (batch + 1) * batch_size
            batch_X = X_train_flat[start_idx:end_idx]
            batch_y = train_dataset_label[start_idx:end_idx]

            _, batch_loss = sess.run([optimizer, loss], feed_dict={input_images: batch_X, input_labels: batch_y})
            avg_loss += batch_loss

        avg_loss /= num_batches
        print(f"Epoch {epoch + 1}/{num_epochs}, Average Loss: {avg_loss}")

    # Evaluation
    test_accuracy = sess.run(accuracy, feed_dict={input_images: X_test_flat, input_labels: test_dataset_label})
    print(f"Test Accuracy: {test_accuracy:.4f}")


In [19]:
# performing predictions on the test dataset
y_pred = svm_model.predict(d2_test_dataset)

print("ACCURACY OF THE MODEL: ", accuracy_score(test_dataset_label, y_pred))

ACCURACY OF THE MODEL:  0.9401041666666666


In [20]:
print(confusion_matrix(test_dataset_label, y_pred))

[[451   0  29   0]
 [  0 446   0  34]
 [ 12   0 468   0]
 [  0  40   0 440]]


In [21]:
svm_poly = SVC(kernel='poly', degree=3)

In [22]:
tic = start_timer()
svm_poly.fit(d2_train_dataset, train_dataset_label)
toc = end_timer()
show_time(tic,toc)

Testing started
Testing time (s) = 4496.371396896997



In [23]:
# performing predictions on the test dataset
y_pred = svm_poly.predict(d2_test_dataset)

print("ACCURACY OF THE MODEL: ", accuracy_score(test_dataset_label, y_pred))

ACCURACY OF THE MODEL:  0.3984375


In [24]:
print(confusion_matrix(test_dataset_label, y_pred))

[[257  85  63  75]
 [135 122 119 104]
 [ 66  91 211 112]
 [ 83  98 124 175]]
